In [48]:
# Imports
import datasets
import pandas as pd

from IPython.display import display, HTML
from datasets import ClassLabel
from transformers import EncoderDecoderModel, BertTokenizerFast
from seq2seq_trainer import Seq2SeqTrainer
from seq2seq_training_args import Seq2SeqTrainingArguments

'''
!pip install tensorflow
!pip install torch===1.7.1 torchvision===0.8.2 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install datasets
!pip install transformers
!pip install git-python==1.0.3
!pip install rouge_score
!pip install sacrebleu
!pip install wget

!python -m wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py
!python -m wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_training_args.py
'''

In [49]:
# Load data
train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")
test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test[:5%]")

train_data.info.description
df = pd.DataFrame(train_data[:1])
del df["id"]

for column, typ in train_data.features.items():
    if isinstance(typ, ClassLabel):
        df[column] = df[column].transform(lambda i: typ.names[i])

display(HTML(df.to_html()))

Reusing dataset cnn_dailymail (C:\Users\Administrator\.cache\huggingface\datasets\cnn_dailymail\3.0.0\3.0.0\0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602)
Reusing dataset cnn_dailymail (C:\Users\Administrator\.cache\huggingface\datasets\cnn_dailymail\3.0.0\3.0.0\0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602)
Reusing dataset cnn_dailymail (C:\Users\Administrator\.cache\huggingface\datasets\cnn_dailymail\3.0.0\3.0.0\0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602)


In [50]:
# Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

In [62]:
# Prepare data
encoder_max_length=512
decoder_max_length=128
batch_size = 16

def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

In [ ]:
# Training data
train_data = train_data.select(range(10000)) # TODO: Update limit

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights", "id"]
)

train_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"]
)

In [64]:
# Validation data
val_data = val_data.select(range(500))

val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    remove_columns=["article", "highlights", "id"]
)

val_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"]
)

In [65]:
# Load models
tf2tf = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-multilingual-cased", "bert-base-multilingual-cased", tie_encoder_decoder=True)
tf2tf.save_pretrained("bert2bert_multilingual_shared_tied")
tf2tf = EncoderDecoderModel.from_pretrained("bert2bert_multilingual_shared_tied")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias

In [66]:
# Configure models
tf2tf.config.decoder_start_token_id = tokenizer.cls_token_id
tf2tf.config.eos_token_id = tokenizer.sep_token_id
tf2tf.config.pad_token_id = tokenizer.pad_token_id
tf2tf.config.vocab_size = tf2tf.config.encoder.vocab_size

In [67]:
# Configure beam search
tf2tf.config.max_length = 142
tf2tf.config.min_length = 56
tf2tf.config.no_repeat_ngram_size = 3
tf2tf.config.early_stopping = True
tf2tf.config.length_penalty = 2.0
tf2tf.config.num_beams = 4

In [71]:
# Prepare metric
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [77]:
!python -m wget 'https://gist.githubusercontent.com/dte/8954e405590a360614dcc6acdb7baa74/raw/d1b5a01ed0b9252654016d2a9a435dc8b4c045e7/install-CUDA-docker-nvidia-docker.sh'

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\wget.py", line 568, in <module>
    filename = download(args[0], out=options.output)
  File "C:\ProgramData\Anaconda3\lib\site-packages\wget.py", line 526, in download
    (tmpfile, headers) = ulib.urlretrieve(binurl, tmpfile, callback)
  File "C:\ProgramData\Anaconda3\lib\urllib\request.py", line 247, in urlretrieve
    with contextlib.closing(urlopen(url, data)) as fp:
  File "C:\ProgramData\Anaconda3\lib\urllib\request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "C:\ProgramData\Anaconda3\lib\urllib\request.py", line 510, in open
    req = Request(fullurl, data)
  File "C:\ProgramData\Anaconda3\lib\urllib\request.py", line 328, in __init__
    self.full_url = url
  F

In [78]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0']


In [80]:
# Setup arguments
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="epoch", # steps
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=False, # True
    output_dir="./",
    warmup_steps=1000, # 2000
    num_train_epochs=3, # 50000
    save_steps=2000,
    logging_steps=2000,
    eval_steps=200, # 2000
    save_total_limit=3
)

In [ ]:
# Start training
trainer = Seq2SeqTrainer(
    model=tf2tf,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)

trainer.train()

C:\ProgramData\Anaconda3\lib\site-packages\datasets\arrow_dataset.py:851: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


Epoch,Training Loss,Validation Loss
